In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weekenddd20/Participants_Data_WH20/Test.csv
/kaggle/input/weekenddd20/Participants_Data_WH20/sample submission.csv
/kaggle/input/weekenddd20/Participants_Data_WH20/Train.csv


In [3]:
train=pd.read_csv('/kaggle/input/weekenddd20/Participants_Data_WH20/Train.csv')
test=pd.read_csv('/kaggle/input/weekenddd20/Participants_Data_WH20/Test.csv')

In [4]:
# Dropping duplicate rows
train=train.drop_duplicates()

In [5]:
#Checking Null values for each column
train.isnull().sum()

Labels      0
Text        0
Text_Tag    2
dtype: int64

In [6]:
# There are only two null values , so i am going to drop the null values.
train=train.dropna()

In [7]:
import nltk
!pip install sentence-transformers

     |████████████████████████████████| 62 kB 474 kB/s eta 0:00:011
     |████████████████████████████████| 884 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 8.1 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-py3-none-any.whl size=101181 sha256=d23638cb0fb1fffc369f1341cb8688e0189930f394dbe09a8f208a8d4e2b0270
  Stored in directory: /root/.cache/pip/wheels/b5/94/b4/953a1fd26652702c88112a188346df4cead56f0e3971a6d653
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will chang

In [8]:
from sentence_transformers import SentenceTransformer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [9]:
embedder = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

100%|██████████| 1.31G/1.31G [04:13<00:00, 5.18MB/s] 


In [10]:
full_df =pd.concat([train, test]).reset_index()

In [11]:
full_df['Length'] = full_df['Text'].apply(len)
full_df['Length1'] = full_df['Text_Tag'].apply(len)

In [12]:
word_embeddings = embedder.encode(full_df.Text)

In [13]:
data = pd.DataFrame(word_embeddings)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,-0.843116,0.354265,0.356840,0.419612,0.502844,0.265882,1.124244,-0.337540,-1.011598,0.389833,...,0.022141,0.510810,0.853409,0.351011,0.043097,0.612750,-0.362310,0.728116,-0.631632,-0.265538
1,-0.154687,0.872684,1.772495,-0.295068,0.222306,-0.253727,-1.300732,0.190506,0.343303,-0.872295,...,-0.493620,0.729953,-0.513199,1.092360,-0.258280,0.267085,-0.399993,-0.463843,-1.466223,0.220742
2,-1.090438,-1.039242,-0.442930,-1.469321,0.165641,-0.411884,-0.524986,1.435754,-0.221386,0.336803,...,0.896936,1.188237,-0.614642,0.053428,-0.562576,0.325781,-1.103370,-1.954508,0.481244,-0.146409
3,0.599439,0.623602,0.633820,0.405360,0.706751,-0.079209,0.344766,-0.220472,-0.569685,-0.079065,...,0.311481,0.002133,-0.285264,0.073709,0.787636,-0.344154,-0.042825,1.363382,0.595174,-0.336082
4,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,...,-0.951390,-0.129189,-0.908807,1.449807,0.192440,0.047139,-0.335581,0.606919,-0.808966,-0.328456


In [14]:
norm_train_reviews =full_df.Text_Tag.values

In [15]:
# I applied countvectorizer on Text_Tag column
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# build BOW features on train reviews
cv = CountVectorizer()
cv_train_features = cv.fit_transform(norm_train_reviews)


# build TFIDF features on train reviews
tv = TfidfVectorizer()
tv_train_features = tv.fit_transform(norm_train_reviews)

In [16]:
cv_matrix = cv_train_features.toarray()

vocab = cv.get_feature_names()
cv=pd.DataFrame(cv_matrix, columns=vocab)

In [17]:
cv.head()

,10,2012,2014,abc,abortion,accomplishments,administration,advertising,afghanistan,after,...,veterans,voting,wall,water,wealth,weather,week,welfare,women,workers
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
cv1=cv.drop(['10'],axis=1)

In [19]:
data12=pd.concat([data,cv1],axis=1)
data12['Labels']=full_df.Labels.values
data12['Length']=full_df.Length.values
data12['Length1']=full_df.Length1.values
data12.head()

,0,1,2,3,4,5,6,7,8,9,...,water,wealth,weather,week,welfare,women,workers,Labels,Length,Length1
0,-0.843116,0.354265,0.356840,0.419612,0.502844,0.265882,1.124244,-0.337540,-1.011598,0.389833,...,0,0,0,0,0,0,0,1.0,82,8
1,-0.154687,0.872684,1.772495,-0.295068,0.222306,-0.253727,-1.300732,0.190506,0.343303,-0.872295,...,0,0,0,0,0,0,0,2.0,141,34
2,-1.090438,-1.039242,-0.442930,-1.469321,0.165641,-0.411884,-0.524986,1.435754,-0.221386,0.336803,...,0,0,0,0,0,0,0,3.0,105,14
3,0.599439,0.623602,0.633820,0.405360,0.706751,-0.079209,0.344766,-0.220472,-0.569685,-0.079065,...,0,0,0,0,0,0,0,1.0,78,11
4,-0.092456,-0.040649,-0.215230,0.439570,1.114174,0.060738,-0.115374,-0.424731,0.706268,-0.155484,...,0,0,0,0,0,0,0,2.0,54,12


In [20]:
train1 = data12[~data12.Labels.isna()]
test1 = data12[data12.Labels.isna()]
test1.drop("Labels",axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
X=train1.drop(['Labels'],axis=1)
y=train1['Labels']

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=1000,task_type='GPU')

In [27]:
import lightgbm as lgb
lgbm=lgb.LGBMClassifier(n_estimators=100,device="gpu",max_bin=25)

In [28]:
from sklearn.ensemble import VotingClassifier
vclf=VotingClassifier(estimators=[('lgg',lgbm),('xgg',model)],voting='soft')

In [29]:
vclf.fit(X_train,y_train)

Learning rate set to 0.099662
0:	learn: 1.7809812	total: 44.7ms	remaining: 44.6s
1:	learn: 1.7701134	total: 68.4ms	remaining: 34.1s
2:	learn: 1.7612158	total: 91.4ms	remaining: 30.4s
3:	learn: 1.7533402	total: 114ms	remaining: 28.3s
4:	learn: 1.7457518	total: 137ms	remaining: 27.3s
5:	learn: 1.7383230	total: 160ms	remaining: 26.5s
6:	learn: 1.7319283	total: 183ms	remaining: 25.9s
7:	learn: 1.7263722	total: 206ms	remaining: 25.6s
8:	learn: 1.7210825	total: 230ms	remaining: 25.3s
9:	learn: 1.7156209	total: 253ms	remaining: 25s
10:	learn: 1.7094844	total: 289ms	remaining: 26s
11:	learn: 1.7043319	total: 316ms	remaining: 26s
12:	learn: 1.6993179	total: 338ms	remaining: 25.7s
13:	learn: 1.6953020	total: 359ms	remaining: 25.3s
14:	learn: 1.6907292	total: 382ms	remaining: 25.1s
15:	learn: 1.6855892	total: 404ms	remaining: 24.8s
16:	learn: 1.6807419	total: 431ms	remaining: 24.9s
17:	learn: 1.6771706	total: 453ms	remaining: 24.7s
18:	learn: 1.6733586	total: 475ms	remaining: 24.5s
19:	learn: 1.6

161:	learn: 1.3179075	total: 3.63s	remaining: 18.8s
162:	learn: 1.3162177	total: 3.66s	remaining: 18.8s
163:	learn: 1.3146944	total: 3.68s	remaining: 18.8s
164:	learn: 1.3130938	total: 3.7s	remaining: 18.7s
165:	learn: 1.3112047	total: 3.72s	remaining: 18.7s
166:	learn: 1.3088411	total: 3.75s	remaining: 18.7s
167:	learn: 1.3069763	total: 3.77s	remaining: 18.7s
168:	learn: 1.3047629	total: 3.79s	remaining: 18.7s
169:	learn: 1.3028424	total: 3.82s	remaining: 18.6s
170:	learn: 1.3008437	total: 3.84s	remaining: 18.6s
171:	learn: 1.2992775	total: 3.86s	remaining: 18.6s
172:	learn: 1.2971154	total: 3.89s	remaining: 18.6s
173:	learn: 1.2950241	total: 3.91s	remaining: 18.6s
174:	learn: 1.2932946	total: 3.93s	remaining: 18.5s
175:	learn: 1.2914823	total: 3.96s	remaining: 18.5s
176:	learn: 1.2890312	total: 3.98s	remaining: 18.5s
177:	learn: 1.2874620	total: 4s	remaining: 18.5s
178:	learn: 1.2853234	total: 4.02s	remaining: 18.5s
179:	learn: 1.2828113	total: 4.05s	remaining: 18.4s
180:	learn: 1.28

320:	learn: 1.0440262	total: 7.75s	remaining: 16.4s
321:	learn: 1.0426796	total: 7.77s	remaining: 16.4s
322:	learn: 1.0412443	total: 7.79s	remaining: 16.3s
323:	learn: 1.0399318	total: 7.81s	remaining: 16.3s
324:	learn: 1.0383573	total: 7.83s	remaining: 16.3s
325:	learn: 1.0366009	total: 7.85s	remaining: 16.2s
326:	learn: 1.0348170	total: 7.88s	remaining: 16.2s
327:	learn: 1.0333114	total: 7.9s	remaining: 16.2s
328:	learn: 1.0315771	total: 7.93s	remaining: 16.2s
329:	learn: 1.0298315	total: 7.95s	remaining: 16.1s
330:	learn: 1.0287981	total: 7.97s	remaining: 16.1s
331:	learn: 1.0266951	total: 7.99s	remaining: 16.1s
332:	learn: 1.0253344	total: 8.02s	remaining: 16.1s
333:	learn: 1.0234722	total: 8.04s	remaining: 16s
334:	learn: 1.0223124	total: 8.06s	remaining: 16s
335:	learn: 1.0210431	total: 8.08s	remaining: 16s
336:	learn: 1.0197928	total: 8.1s	remaining: 15.9s
337:	learn: 1.0184910	total: 8.12s	remaining: 15.9s
338:	learn: 1.0168516	total: 8.15s	remaining: 15.9s
339:	learn: 1.015501

479:	learn: 0.8359379	total: 11.2s	remaining: 12.1s
480:	learn: 0.8348167	total: 11.2s	remaining: 12.1s
481:	learn: 0.8336469	total: 11.2s	remaining: 12.1s
482:	learn: 0.8324441	total: 11.3s	remaining: 12.1s
483:	learn: 0.8307415	total: 11.3s	remaining: 12s
484:	learn: 0.8298971	total: 11.3s	remaining: 12s
485:	learn: 0.8285679	total: 11.3s	remaining: 12s
486:	learn: 0.8273682	total: 11.4s	remaining: 12s
487:	learn: 0.8259962	total: 11.4s	remaining: 11.9s
488:	learn: 0.8247226	total: 11.4s	remaining: 11.9s
489:	learn: 0.8239876	total: 11.4s	remaining: 11.9s
490:	learn: 0.8230108	total: 11.4s	remaining: 11.9s
491:	learn: 0.8219970	total: 11.5s	remaining: 11.8s
492:	learn: 0.8206714	total: 11.5s	remaining: 11.8s
493:	learn: 0.8198229	total: 11.5s	remaining: 11.8s
494:	learn: 0.8185135	total: 11.5s	remaining: 11.8s
495:	learn: 0.8172064	total: 11.6s	remaining: 11.7s
496:	learn: 0.8158199	total: 11.6s	remaining: 11.7s
497:	learn: 0.8145040	total: 11.6s	remaining: 11.7s
498:	learn: 0.813117

638:	learn: 0.6768592	total: 14.6s	remaining: 8.27s
639:	learn: 0.6762837	total: 14.7s	remaining: 8.25s
640:	learn: 0.6753878	total: 14.7s	remaining: 8.22s
641:	learn: 0.6748202	total: 14.7s	remaining: 8.2s
642:	learn: 0.6738574	total: 14.7s	remaining: 8.18s
643:	learn: 0.6728090	total: 14.7s	remaining: 8.15s
644:	learn: 0.6717548	total: 14.8s	remaining: 8.13s
645:	learn: 0.6706655	total: 14.8s	remaining: 8.1s
646:	learn: 0.6696697	total: 14.8s	remaining: 8.08s
647:	learn: 0.6690125	total: 14.8s	remaining: 8.06s
648:	learn: 0.6682409	total: 14.9s	remaining: 8.03s
649:	learn: 0.6672589	total: 14.9s	remaining: 8.01s
650:	learn: 0.6664504	total: 14.9s	remaining: 7.99s
651:	learn: 0.6657088	total: 14.9s	remaining: 7.96s
652:	learn: 0.6645164	total: 14.9s	remaining: 7.94s
653:	learn: 0.6637229	total: 15s	remaining: 7.92s
654:	learn: 0.6630425	total: 15s	remaining: 7.89s
655:	learn: 0.6621410	total: 15s	remaining: 7.87s
656:	learn: 0.6612993	total: 15s	remaining: 7.84s
657:	learn: 0.6604947	

797:	learn: 0.5531662	total: 18.9s	remaining: 4.78s
798:	learn: 0.5523942	total: 18.9s	remaining: 4.75s
799:	learn: 0.5513619	total: 18.9s	remaining: 4.73s
800:	learn: 0.5504719	total: 18.9s	remaining: 4.7s
801:	learn: 0.5496991	total: 19s	remaining: 4.68s
802:	learn: 0.5492408	total: 19s	remaining: 4.66s
803:	learn: 0.5486852	total: 19s	remaining: 4.63s
804:	learn: 0.5480326	total: 19s	remaining: 4.61s
805:	learn: 0.5472207	total: 19s	remaining: 4.58s
806:	learn: 0.5467329	total: 19.1s	remaining: 4.56s
807:	learn: 0.5463460	total: 19.1s	remaining: 4.53s
808:	learn: 0.5459844	total: 19.1s	remaining: 4.51s
809:	learn: 0.5454855	total: 19.1s	remaining: 4.49s
810:	learn: 0.5446745	total: 19.1s	remaining: 4.46s
811:	learn: 0.5441738	total: 19.2s	remaining: 4.44s
812:	learn: 0.5434836	total: 19.2s	remaining: 4.41s
813:	learn: 0.5429334	total: 19.2s	remaining: 4.39s
814:	learn: 0.5420258	total: 19.2s	remaining: 4.36s
815:	learn: 0.5412776	total: 19.2s	remaining: 4.34s
816:	learn: 0.5405178	t

956:	learn: 0.4548837	total: 22.2s	remaining: 998ms
957:	learn: 0.4544664	total: 22.2s	remaining: 975ms
958:	learn: 0.4539495	total: 22.2s	remaining: 951ms
959:	learn: 0.4533776	total: 22.3s	remaining: 928ms
960:	learn: 0.4526379	total: 22.3s	remaining: 905ms
961:	learn: 0.4520217	total: 22.3s	remaining: 881ms
962:	learn: 0.4513597	total: 22.3s	remaining: 858ms
963:	learn: 0.4507824	total: 22.4s	remaining: 835ms
964:	learn: 0.4502653	total: 22.4s	remaining: 812ms
965:	learn: 0.4495163	total: 22.4s	remaining: 788ms
966:	learn: 0.4491777	total: 22.4s	remaining: 765ms
967:	learn: 0.4487436	total: 22.4s	remaining: 742ms
968:	learn: 0.4481972	total: 22.5s	remaining: 718ms
969:	learn: 0.4475662	total: 22.5s	remaining: 695ms
970:	learn: 0.4470749	total: 22.5s	remaining: 672ms
971:	learn: 0.4466117	total: 22.5s	remaining: 649ms
972:	learn: 0.4459713	total: 22.5s	remaining: 626ms
973:	learn: 0.4456924	total: 22.6s	remaining: 602ms
974:	learn: 0.4452147	total: 22.6s	remaining: 579ms
975:	learn: 

VotingClassifier(estimators=[('lgg', LGBMClassifier(device='gpu', max_bin=25)),
                             ('xgg',
                              <catboost.core.CatBoostClassifier object at 0x7fb16feba2d0>)],
                 voting='soft')

In [30]:
prediction=vclf.predict_proba(test1)

In [31]:
# Results are using Robert large sentence transformer
results=pd.DataFrame(prediction, columns=[0,1,2,3,4,5])

In [32]:
embedder1 = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

100%|██████████| 459M/459M [00:19<00:00, 24.0MB/s] 


In [33]:
%%time
base_embeddings=embedder1.encode(full_df.Text.values.tolist(),batch_size=128,show_progress_bar=True)


CPU times: user 10.4 s, sys: 1.38 s, total: 11.8 s
Wall time: 11.9 s


In [34]:
data= pd.DataFrame(base_embeddings)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.063559,0.073300,-0.032389,1.328562,-0.049966,0.453855,0.628987,0.656944,0.336365,-0.718979,...,0.707739,0.125505,0.349462,-0.382211,1.345453,0.671241,-0.090033,-0.924622,0.593463,-0.151931
1,-0.251282,-0.427436,-0.002071,-1.095850,-0.286984,-0.405636,0.545493,-0.388819,0.524717,0.264293,...,0.449813,-1.002035,-0.211106,0.256451,-0.323354,0.298147,0.432782,-1.182147,0.014275,-0.618577
2,0.573638,0.121656,-0.146542,0.829190,-0.009897,-0.721362,0.463740,0.480717,1.374566,-0.134241,...,-0.635795,-0.543450,-1.251009,0.389167,0.127668,-0.334683,0.480950,0.778091,-0.727586,0.076265
3,1.256146,0.762251,0.183387,-0.102740,0.100235,0.485516,0.039702,-0.064118,-0.129972,-0.187275,...,0.355464,-0.488293,0.624699,-0.668803,0.678824,0.429867,0.099317,-1.353077,0.240512,-0.293187
4,-0.105395,0.239276,0.312050,-0.679842,-0.468682,-0.453707,0.445754,-0.465910,0.302358,0.581789,...,-0.121204,-0.133457,-0.121812,1.138613,0.084732,0.470364,-0.812005,0.102160,0.069002,-0.161689


In [35]:
data12=pd.concat([data,cv1],axis=1)
data12['Labels']=full_df.Labels.values
data12['Length']=full_df.Length.values
data12['Length1']=full_df.Length1.values
data12.head()

,0,1,2,3,4,5,6,7,8,9,...,water,wealth,weather,week,welfare,women,workers,Labels,Length,Length1
0,0.063559,0.073300,-0.032389,1.328562,-0.049966,0.453855,0.628987,0.656944,0.336365,-0.718979,...,0,0,0,0,0,0,0,1.0,82,8
1,-0.251282,-0.427436,-0.002071,-1.095850,-0.286984,-0.405636,0.545493,-0.388819,0.524717,0.264293,...,0,0,0,0,0,0,0,2.0,141,34
2,0.573638,0.121656,-0.146542,0.829190,-0.009897,-0.721362,0.463740,0.480717,1.374566,-0.134241,...,0,0,0,0,0,0,0,3.0,105,14
3,1.256146,0.762251,0.183387,-0.102740,0.100235,0.485516,0.039702,-0.064118,-0.129972,-0.187275,...,0,0,0,0,0,0,0,1.0,78,11
4,-0.105395,0.239276,0.312050,-0.679842,-0.468682,-0.453707,0.445754,-0.465910,0.302358,0.581789,...,0,0,0,0,0,0,0,2.0,54,12


In [36]:
train1 = data12[~data12.Labels.isna()]
test1 = data12[data12.Labels.isna()]
test1.drop("Labels",axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
X=train1.drop(['Labels'],axis=1)
y=train1['Labels']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [40]:
vclf.fit(X_train,y_train)

Learning rate set to 0.099662


0:	learn: 1.7807814	total: 24.7ms	remaining: 24.6s
1:	learn: 1.7701706	total: 46.8ms	remaining: 23.4s
2:	learn: 1.7620153	total: 68ms	remaining: 22.6s
3:	learn: 1.7533545	total: 89ms	remaining: 22.2s
4:	learn: 1.7457036	total: 111ms	remaining: 22s
5:	learn: 1.7389478	total: 132ms	remaining: 21.8s
6:	learn: 1.7326926	total: 156ms	remaining: 22.2s
7:	learn: 1.7263871	total: 176ms	remaining: 21.9s
8:	learn: 1.7213131	total: 196ms	remaining: 21.6s
9:	learn: 1.7151239	total: 216ms	remaining: 21.4s
10:	learn: 1.7092536	total: 237ms	remaining: 21.3s
11:	learn: 1.7047622	total: 257ms	remaining: 21.2s
12:	learn: 1.6992748	total: 278ms	remaining: 21.1s
13:	learn: 1.6952160	total: 297ms	remaining: 20.9s
14:	learn: 1.6916305	total: 323ms	remaining: 21.2s
15:	learn: 1.6878414	total: 342ms	remaining: 21.1s
16:	learn: 1.6833278	total: 363ms	remaining: 21s
17:	learn: 1.6793347	total: 384ms	remaining: 20.9s
18:	learn: 1.6746868	total: 404ms	remaining: 20.9s
19:	learn: 1.6713466	total: 424ms	remaining: 

161:	learn: 1.3353711	total: 3.06s	remaining: 15.8s
162:	learn: 1.3335663	total: 3.08s	remaining: 15.8s
163:	learn: 1.3314540	total: 3.1s	remaining: 15.8s
164:	learn: 1.3290703	total: 3.12s	remaining: 15.8s
165:	learn: 1.3265047	total: 3.14s	remaining: 15.8s
166:	learn: 1.3239763	total: 3.16s	remaining: 15.7s
167:	learn: 1.3218940	total: 3.17s	remaining: 15.7s
168:	learn: 1.3201810	total: 3.19s	remaining: 15.7s
169:	learn: 1.3186669	total: 3.21s	remaining: 15.7s
170:	learn: 1.3167036	total: 3.23s	remaining: 15.6s
171:	learn: 1.3142420	total: 3.25s	remaining: 15.6s
172:	learn: 1.3122139	total: 3.26s	remaining: 15.6s
173:	learn: 1.3100274	total: 3.28s	remaining: 15.6s
174:	learn: 1.3082204	total: 3.3s	remaining: 15.6s
175:	learn: 1.3059796	total: 3.32s	remaining: 15.5s
176:	learn: 1.3043356	total: 3.34s	remaining: 15.5s
177:	learn: 1.3021187	total: 3.35s	remaining: 15.5s
178:	learn: 1.3006714	total: 3.37s	remaining: 15.5s
179:	learn: 1.2990279	total: 3.39s	remaining: 15.4s
180:	learn: 1.

320:	learn: 1.0523660	total: 6.49s	remaining: 13.7s
321:	learn: 1.0510028	total: 6.51s	remaining: 13.7s
322:	learn: 1.0497880	total: 6.54s	remaining: 13.7s
323:	learn: 1.0481085	total: 6.56s	remaining: 13.7s
324:	learn: 1.0466435	total: 6.58s	remaining: 13.7s
325:	learn: 1.0452868	total: 6.6s	remaining: 13.6s
326:	learn: 1.0439263	total: 6.62s	remaining: 13.6s
327:	learn: 1.0423339	total: 6.64s	remaining: 13.6s
328:	learn: 1.0408393	total: 6.66s	remaining: 13.6s
329:	learn: 1.0394241	total: 6.68s	remaining: 13.6s
330:	learn: 1.0383414	total: 6.7s	remaining: 13.6s
331:	learn: 1.0374148	total: 6.72s	remaining: 13.5s
332:	learn: 1.0363277	total: 6.74s	remaining: 13.5s
333:	learn: 1.0350039	total: 6.77s	remaining: 13.5s
334:	learn: 1.0333281	total: 6.79s	remaining: 13.5s
335:	learn: 1.0316748	total: 6.81s	remaining: 13.5s
336:	learn: 1.0302572	total: 6.83s	remaining: 13.4s
337:	learn: 1.0291445	total: 6.85s	remaining: 13.4s
338:	learn: 1.0275476	total: 6.87s	remaining: 13.4s
339:	learn: 1.

479:	learn: 0.8479447	total: 9.8s	remaining: 10.6s
480:	learn: 0.8470386	total: 9.82s	remaining: 10.6s
481:	learn: 0.8458970	total: 9.84s	remaining: 10.6s
482:	learn: 0.8449716	total: 9.86s	remaining: 10.6s
483:	learn: 0.8437493	total: 9.88s	remaining: 10.5s
484:	learn: 0.8427711	total: 9.9s	remaining: 10.5s
485:	learn: 0.8414921	total: 9.92s	remaining: 10.5s
486:	learn: 0.8405510	total: 9.94s	remaining: 10.5s
487:	learn: 0.8396166	total: 9.96s	remaining: 10.4s
488:	learn: 0.8381270	total: 9.98s	remaining: 10.4s
489:	learn: 0.8373244	total: 10s	remaining: 10.4s
490:	learn: 0.8361493	total: 10s	remaining: 10.4s
491:	learn: 0.8347686	total: 10s	remaining: 10.4s
492:	learn: 0.8336805	total: 10.1s	remaining: 10.3s
493:	learn: 0.8326010	total: 10.1s	remaining: 10.3s
494:	learn: 0.8312358	total: 10.1s	remaining: 10.3s
495:	learn: 0.8302384	total: 10.1s	remaining: 10.3s
496:	learn: 0.8290640	total: 10.1s	remaining: 10.3s
497:	learn: 0.8274281	total: 10.2s	remaining: 10.2s
498:	learn: 0.826395

638:	learn: 0.6838008	total: 13s	remaining: 7.37s
639:	learn: 0.6828509	total: 13.1s	remaining: 7.35s
640:	learn: 0.6818586	total: 13.1s	remaining: 7.33s
641:	learn: 0.6812251	total: 13.1s	remaining: 7.31s
642:	learn: 0.6797849	total: 13.1s	remaining: 7.29s
643:	learn: 0.6788351	total: 13.1s	remaining: 7.27s
644:	learn: 0.6777950	total: 13.2s	remaining: 7.25s
645:	learn: 0.6768696	total: 13.2s	remaining: 7.23s
646:	learn: 0.6757416	total: 13.2s	remaining: 7.21s
647:	learn: 0.6749794	total: 13.2s	remaining: 7.19s
648:	learn: 0.6739445	total: 13.3s	remaining: 7.17s
649:	learn: 0.6731408	total: 13.3s	remaining: 7.15s
650:	learn: 0.6721138	total: 13.3s	remaining: 7.13s
651:	learn: 0.6712052	total: 13.3s	remaining: 7.11s
652:	learn: 0.6703114	total: 13.3s	remaining: 7.08s
653:	learn: 0.6691797	total: 13.4s	remaining: 7.07s
654:	learn: 0.6683353	total: 13.4s	remaining: 7.04s
655:	learn: 0.6676915	total: 13.4s	remaining: 7.02s
656:	learn: 0.6670484	total: 13.4s	remaining: 7s
657:	learn: 0.666

797:	learn: 0.5553066	total: 16.7s	remaining: 4.23s
798:	learn: 0.5547339	total: 16.7s	remaining: 4.21s
799:	learn: 0.5538080	total: 16.8s	remaining: 4.19s
800:	learn: 0.5530641	total: 16.8s	remaining: 4.17s
801:	learn: 0.5521452	total: 16.8s	remaining: 4.14s
802:	learn: 0.5513552	total: 16.8s	remaining: 4.12s
803:	learn: 0.5505936	total: 16.8s	remaining: 4.1s
804:	learn: 0.5501470	total: 16.8s	remaining: 4.08s
805:	learn: 0.5493358	total: 16.9s	remaining: 4.06s
806:	learn: 0.5485996	total: 16.9s	remaining: 4.04s
807:	learn: 0.5481334	total: 16.9s	remaining: 4.02s
808:	learn: 0.5473052	total: 16.9s	remaining: 4s
809:	learn: 0.5467905	total: 16.9s	remaining: 3.97s
810:	learn: 0.5459766	total: 17s	remaining: 3.95s
811:	learn: 0.5452699	total: 17s	remaining: 3.93s
812:	learn: 0.5445981	total: 17s	remaining: 3.91s
813:	learn: 0.5439556	total: 17s	remaining: 3.89s
814:	learn: 0.5431876	total: 17s	remaining: 3.87s
815:	learn: 0.5425648	total: 17.1s	remaining: 3.85s
816:	learn: 0.5419427	tota

956:	learn: 0.4559569	total: 19.7s	remaining: 883ms
957:	learn: 0.4553731	total: 19.7s	remaining: 862ms
958:	learn: 0.4549020	total: 19.7s	remaining: 842ms
959:	learn: 0.4542812	total: 19.7s	remaining: 821ms
960:	learn: 0.4536988	total: 19.7s	remaining: 801ms
961:	learn: 0.4530708	total: 19.7s	remaining: 780ms
962:	learn: 0.4524617	total: 19.8s	remaining: 759ms
963:	learn: 0.4516072	total: 19.8s	remaining: 739ms
964:	learn: 0.4513700	total: 19.8s	remaining: 718ms
965:	learn: 0.4507290	total: 19.8s	remaining: 698ms
966:	learn: 0.4500566	total: 19.8s	remaining: 677ms
967:	learn: 0.4493739	total: 19.9s	remaining: 657ms
968:	learn: 0.4486651	total: 19.9s	remaining: 636ms
969:	learn: 0.4480843	total: 19.9s	remaining: 616ms
970:	learn: 0.4476769	total: 19.9s	remaining: 595ms
971:	learn: 0.4472051	total: 19.9s	remaining: 574ms
972:	learn: 0.4466827	total: 20s	remaining: 554ms
973:	learn: 0.4462121	total: 20s	remaining: 533ms
974:	learn: 0.4456107	total: 20s	remaining: 513ms
975:	learn: 0.4452

VotingClassifier(estimators=[('lgg', LGBMClassifier(device='gpu', max_bin=25)),
                             ('xgg',
                              <catboost.core.CatBoostClassifier object at 0x7fb16feba2d0>)],
                 voting='soft')

In [41]:
prediction=vclf.predict_proba(test1)

In [42]:
# Results are using Robert base sentence transformer
results1=pd.DataFrame(prediction, columns=[0,1,2,3,4,5])

In [43]:
embedder2 = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

100%|██████████| 1.24G/1.24G [00:58<00:00, 21.2MB/s] 


In [44]:
word_embeddings1 = embedder2.encode(full_df.Text)

In [45]:
data=pd.DataFrame(word_embeddings1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.700180,-0.769513,-0.544782,-0.618669,-0.919392,0.513581,0.670450,0.689292,0.456358,-0.047067,...,0.039128,-0.341870,-0.299323,-0.543496,0.348448,-1.271895,-0.134442,-1.123390,0.452224,0.162756
1,-0.108184,-0.801630,0.147472,-0.389542,0.408265,0.538518,0.736174,-0.005731,-0.525059,-0.159115,...,-0.004297,-0.261957,-0.488298,0.085653,-0.105378,0.064214,1.115160,0.133910,-0.467474,-0.955050
2,-0.344798,-0.046441,0.021711,-0.201737,-0.174805,1.339390,-0.052499,0.256828,-0.738929,0.202591,...,0.163834,0.541816,-0.641019,-1.710618,0.454957,-0.250923,0.295782,-0.131281,0.645660,0.212365
3,0.834027,-0.505015,-0.110948,-0.977177,-0.278406,-0.174256,-0.116903,0.578716,-0.399319,-0.425492,...,0.047619,-1.227428,-0.076522,-0.311735,0.856727,-0.444029,-1.038579,-1.068880,0.275600,0.559146
4,0.542017,0.590681,-0.069034,-1.087724,-0.278888,0.354331,0.212271,0.297291,-0.337527,-0.086165,...,-0.170516,-0.868259,-0.413786,0.305759,-0.469030,-0.374569,0.650067,0.069905,-0.076002,0.253064


In [46]:
data12=pd.concat([data,cv1],axis=1)
data12['Labels']=full_df.Labels.values
data12['Length']=full_df.Length.values
data12['Length1']=full_df.Length1.values
data12.head()

,0,1,2,3,4,5,6,7,8,9,...,water,wealth,weather,week,welfare,women,workers,Labels,Length,Length1
0,0.700180,-0.769513,-0.544782,-0.618669,-0.919392,0.513581,0.670450,0.689292,0.456358,-0.047067,...,0,0,0,0,0,0,0,1.0,82,8
1,-0.108184,-0.801630,0.147472,-0.389542,0.408265,0.538518,0.736174,-0.005731,-0.525059,-0.159115,...,0,0,0,0,0,0,0,2.0,141,34
2,-0.344798,-0.046441,0.021711,-0.201737,-0.174805,1.339390,-0.052499,0.256828,-0.738929,0.202591,...,0,0,0,0,0,0,0,3.0,105,14
3,0.834027,-0.505015,-0.110948,-0.977177,-0.278406,-0.174256,-0.116903,0.578716,-0.399319,-0.425492,...,0,0,0,0,0,0,0,1.0,78,11
4,0.542017,0.590681,-0.069034,-1.087724,-0.278888,0.354331,0.212271,0.297291,-0.337527,-0.086165,...,0,0,0,0,0,0,0,2.0,54,12


In [47]:
train1 = data12[~data12.Labels.isna()]
test1 = data12[data12.Labels.isna()]
test1.drop("Labels",axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [48]:
X=train1.drop(['Labels'],axis=1)
y=train1['Labels']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [50]:
vclf.fit(X_train,y_train)

Learning rate set to 0.099662


0:	learn: 1.7804193	total: 30ms	remaining: 29.9s
1:	learn: 1.7699475	total: 53.8ms	remaining: 26.8s
2:	learn: 1.7618528	total: 77.3ms	remaining: 25.7s
3:	learn: 1.7536882	total: 101ms	remaining: 25.1s
4:	learn: 1.7455094	total: 125ms	remaining: 24.8s
5:	learn: 1.7383176	total: 152ms	remaining: 25.1s
6:	learn: 1.7311625	total: 174ms	remaining: 24.7s
7:	learn: 1.7252502	total: 197ms	remaining: 24.4s
8:	learn: 1.7198398	total: 219ms	remaining: 24.1s
9:	learn: 1.7140081	total: 242ms	remaining: 23.9s
10:	learn: 1.7090032	total: 265ms	remaining: 23.8s
11:	learn: 1.7044861	total: 287ms	remaining: 23.6s
12:	learn: 1.6995858	total: 309ms	remaining: 23.5s
13:	learn: 1.6953717	total: 330ms	remaining: 23.3s
14:	learn: 1.6910792	total: 352ms	remaining: 23.1s
15:	learn: 1.6869751	total: 374ms	remaining: 23s
16:	learn: 1.6832760	total: 396ms	remaining: 22.9s
17:	learn: 1.6797771	total: 418ms	remaining: 22.8s
18:	learn: 1.6765044	total: 440ms	remaining: 22.7s
19:	learn: 1.6723955	total: 463ms	remainin

161:	learn: 1.3182614	total: 3.6s	remaining: 18.6s
162:	learn: 1.3154429	total: 3.62s	remaining: 18.6s
163:	learn: 1.3133165	total: 3.64s	remaining: 18.6s
164:	learn: 1.3107716	total: 3.66s	remaining: 18.5s
165:	learn: 1.3090590	total: 3.69s	remaining: 18.5s
166:	learn: 1.3078422	total: 3.71s	remaining: 18.5s
167:	learn: 1.3049236	total: 3.73s	remaining: 18.5s
168:	learn: 1.3031261	total: 3.75s	remaining: 18.4s
169:	learn: 1.3011439	total: 3.77s	remaining: 18.4s
170:	learn: 1.2993793	total: 3.79s	remaining: 18.4s
171:	learn: 1.2970230	total: 3.81s	remaining: 18.4s
172:	learn: 1.2952498	total: 3.83s	remaining: 18.3s
173:	learn: 1.2931196	total: 3.86s	remaining: 18.3s
174:	learn: 1.2908786	total: 3.88s	remaining: 18.3s
175:	learn: 1.2883895	total: 3.9s	remaining: 18.3s
176:	learn: 1.2871196	total: 3.94s	remaining: 18.3s
177:	learn: 1.2849333	total: 3.97s	remaining: 18.3s
178:	learn: 1.2825965	total: 4.04s	remaining: 18.6s
179:	learn: 1.2802620	total: 4.08s	remaining: 18.6s
180:	learn: 1.

320:	learn: 1.0393202	total: 7.51s	remaining: 15.9s
321:	learn: 1.0376078	total: 7.53s	remaining: 15.9s
322:	learn: 1.0358927	total: 7.56s	remaining: 15.8s
323:	learn: 1.0345875	total: 7.58s	remaining: 15.8s
324:	learn: 1.0329845	total: 7.6s	remaining: 15.8s
325:	learn: 1.0318995	total: 7.63s	remaining: 15.8s
326:	learn: 1.0298398	total: 7.65s	remaining: 15.7s
327:	learn: 1.0282881	total: 7.67s	remaining: 15.7s
328:	learn: 1.0262283	total: 7.7s	remaining: 15.7s
329:	learn: 1.0247913	total: 7.72s	remaining: 15.7s
330:	learn: 1.0236828	total: 7.74s	remaining: 15.7s
331:	learn: 1.0218144	total: 7.77s	remaining: 15.6s
332:	learn: 1.0204655	total: 7.79s	remaining: 15.6s
333:	learn: 1.0196068	total: 7.81s	remaining: 15.6s
334:	learn: 1.0182732	total: 7.84s	remaining: 15.6s
335:	learn: 1.0167235	total: 7.86s	remaining: 15.5s
336:	learn: 1.0154350	total: 7.88s	remaining: 15.5s
337:	learn: 1.0139914	total: 7.91s	remaining: 15.5s
338:	learn: 1.0125213	total: 7.93s	remaining: 15.5s
339:	learn: 1.

479:	learn: 0.8348649	total: 11.3s	remaining: 12.2s
480:	learn: 0.8336930	total: 11.3s	remaining: 12.2s
481:	learn: 0.8324175	total: 11.3s	remaining: 12.2s
482:	learn: 0.8314369	total: 11.3s	remaining: 12.1s
483:	learn: 0.8303511	total: 11.4s	remaining: 12.1s
484:	learn: 0.8294702	total: 11.4s	remaining: 12.1s
485:	learn: 0.8281848	total: 11.4s	remaining: 12.1s
486:	learn: 0.8268986	total: 11.4s	remaining: 12s
487:	learn: 0.8258886	total: 11.5s	remaining: 12s
488:	learn: 0.8244948	total: 11.5s	remaining: 12s
489:	learn: 0.8230612	total: 11.5s	remaining: 12s
490:	learn: 0.8221504	total: 11.5s	remaining: 12s
491:	learn: 0.8209233	total: 11.6s	remaining: 11.9s
492:	learn: 0.8196752	total: 11.6s	remaining: 11.9s
493:	learn: 0.8185415	total: 11.6s	remaining: 11.9s
494:	learn: 0.8174083	total: 11.6s	remaining: 11.9s
495:	learn: 0.8162475	total: 11.6s	remaining: 11.8s
496:	learn: 0.8148539	total: 11.7s	remaining: 11.8s
497:	learn: 0.8138986	total: 11.7s	remaining: 11.8s
498:	learn: 0.8125869	

638:	learn: 0.6722502	total: 15.1s	remaining: 8.53s
639:	learn: 0.6715887	total: 15.1s	remaining: 8.51s
640:	learn: 0.6706070	total: 15.1s	remaining: 8.48s
641:	learn: 0.6694422	total: 15.2s	remaining: 8.46s
642:	learn: 0.6686975	total: 15.2s	remaining: 8.43s
643:	learn: 0.6675424	total: 15.2s	remaining: 8.41s
644:	learn: 0.6668269	total: 15.2s	remaining: 8.38s
645:	learn: 0.6659481	total: 15.3s	remaining: 8.36s
646:	learn: 0.6650849	total: 15.3s	remaining: 8.33s
647:	learn: 0.6641928	total: 15.3s	remaining: 8.31s
648:	learn: 0.6630981	total: 15.3s	remaining: 8.28s
649:	learn: 0.6621240	total: 15.3s	remaining: 8.26s
650:	learn: 0.6612368	total: 15.4s	remaining: 8.23s
651:	learn: 0.6602761	total: 15.4s	remaining: 8.21s
652:	learn: 0.6595174	total: 15.4s	remaining: 8.18s
653:	learn: 0.6586300	total: 15.4s	remaining: 8.16s
654:	learn: 0.6580362	total: 15.4s	remaining: 8.13s
655:	learn: 0.6574502	total: 15.5s	remaining: 8.11s
656:	learn: 0.6566312	total: 15.5s	remaining: 8.09s
657:	learn: 

797:	learn: 0.5479210	total: 19.2s	remaining: 4.86s
798:	learn: 0.5472000	total: 19.2s	remaining: 4.83s
799:	learn: 0.5466899	total: 19.2s	remaining: 4.81s
800:	learn: 0.5461227	total: 19.3s	remaining: 4.78s
801:	learn: 0.5450176	total: 19.3s	remaining: 4.76s
802:	learn: 0.5444283	total: 19.3s	remaining: 4.74s
803:	learn: 0.5437725	total: 19.3s	remaining: 4.71s
804:	learn: 0.5430779	total: 19.3s	remaining: 4.68s
805:	learn: 0.5422289	total: 19.4s	remaining: 4.66s
806:	learn: 0.5415695	total: 19.4s	remaining: 4.64s
807:	learn: 0.5409576	total: 19.4s	remaining: 4.62s
808:	learn: 0.5403271	total: 19.5s	remaining: 4.59s
809:	learn: 0.5398316	total: 19.5s	remaining: 4.57s
810:	learn: 0.5391353	total: 19.5s	remaining: 4.54s
811:	learn: 0.5383677	total: 19.5s	remaining: 4.52s
812:	learn: 0.5375733	total: 19.5s	remaining: 4.5s
813:	learn: 0.5366055	total: 19.6s	remaining: 4.47s
814:	learn: 0.5361927	total: 19.6s	remaining: 4.45s
815:	learn: 0.5353065	total: 19.6s	remaining: 4.42s
816:	learn: 0

956:	learn: 0.4501204	total: 23.1s	remaining: 1.04s
957:	learn: 0.4496008	total: 23.1s	remaining: 1.01s
958:	learn: 0.4490029	total: 23.1s	remaining: 989ms
959:	learn: 0.4484263	total: 23.2s	remaining: 965ms
960:	learn: 0.4480368	total: 23.2s	remaining: 941ms
961:	learn: 0.4474896	total: 23.2s	remaining: 917ms
962:	learn: 0.4469857	total: 23.2s	remaining: 892ms
963:	learn: 0.4462390	total: 23.2s	remaining: 868ms
964:	learn: 0.4456792	total: 23.3s	remaining: 844ms
965:	learn: 0.4450585	total: 23.3s	remaining: 820ms
966:	learn: 0.4443923	total: 23.3s	remaining: 796ms
967:	learn: 0.4439423	total: 23.3s	remaining: 771ms
968:	learn: 0.4434217	total: 23.4s	remaining: 747ms
969:	learn: 0.4428169	total: 23.4s	remaining: 723ms
970:	learn: 0.4421670	total: 23.4s	remaining: 699ms
971:	learn: 0.4414142	total: 23.4s	remaining: 674ms
972:	learn: 0.4406751	total: 23.4s	remaining: 650ms
973:	learn: 0.4399620	total: 23.5s	remaining: 626ms
974:	learn: 0.4394301	total: 23.5s	remaining: 602ms
975:	learn: 

VotingClassifier(estimators=[('lgg', LGBMClassifier(device='gpu', max_bin=25)),
                             ('xgg',
                              <catboost.core.CatBoostClassifier object at 0x7fb16feba2d0>)],
                 voting='soft')

In [51]:
prediction=vclf.predict_proba(test1)

In [52]:
# Results are using bert large sentence transformer
results2=pd.DataFrame(prediction, columns=[0,1,2,3,4,5])

In [53]:
embedder3= SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

100%|██████████| 245M/245M [00:12<00:00, 20.3MB/s] 


In [54]:
%%time
base_embeddings2=embedder3.encode(full_df.Text.values.tolist(),batch_size=128,show_progress_bar=True)


CPU times: user 10.4 s, sys: 173 ms, total: 10.6 s
Wall time: 11 s


In [55]:
data=pd.DataFrame(base_embeddings2)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.144368,-0.238294,0.599350,0.149775,0.170297,-0.141637,0.989932,-0.510203,0.628405,0.283468,...,-1.966928,-1.018470,-0.524895,0.260997,-0.420094,0.543894,1.047531,0.079142,-0.001416,-0.297475
1,0.296415,-0.713782,-0.306153,-0.011605,0.344352,-1.188767,0.150497,-0.162290,0.770730,-0.092008,...,0.748893,-0.861240,0.282171,-0.387093,-0.145990,-0.689599,0.727035,0.093808,0.613867,-0.156318
2,-0.440064,0.455458,-0.893356,0.295065,0.160187,-0.527492,0.092013,-0.255349,0.982621,0.172117,...,-0.485759,-0.251611,0.203046,0.436484,0.584199,0.138801,0.179525,0.117047,-0.514392,-0.121610
3,0.088425,-0.284115,0.300834,-0.170313,-0.476598,-0.773967,0.929880,-0.669233,1.217801,-0.511516,...,-0.428810,-0.814650,0.189426,0.598014,-0.078981,0.344360,0.305921,0.111837,-0.194737,0.286152
4,-0.585023,-0.932097,0.649800,-1.510238,0.908814,0.078908,-0.227934,-0.126206,-0.677687,-0.472893,...,-0.032966,-0.012548,1.557752,0.467225,0.275561,-0.823539,-0.176462,0.279183,-0.277266,-0.130289


In [56]:
data12=pd.concat([data,cv1],axis=1)
data12['Labels']=full_df.Labels.values
data12['Length']=full_df.Length.values
data12['Length1']=full_df.Length1.values
data12.head()

,0,1,2,3,4,5,6,7,8,9,...,water,wealth,weather,week,welfare,women,workers,Labels,Length,Length1
0,-0.144368,-0.238294,0.599350,0.149775,0.170297,-0.141637,0.989932,-0.510203,0.628405,0.283468,...,0,0,0,0,0,0,0,1.0,82,8
1,0.296415,-0.713782,-0.306153,-0.011605,0.344352,-1.188767,0.150497,-0.162290,0.770730,-0.092008,...,0,0,0,0,0,0,0,2.0,141,34
2,-0.440064,0.455458,-0.893356,0.295065,0.160187,-0.527492,0.092013,-0.255349,0.982621,0.172117,...,0,0,0,0,0,0,0,3.0,105,14
3,0.088425,-0.284115,0.300834,-0.170313,-0.476598,-0.773967,0.929880,-0.669233,1.217801,-0.511516,...,0,0,0,0,0,0,0,1.0,78,11
4,-0.585023,-0.932097,0.649800,-1.510238,0.908814,0.078908,-0.227934,-0.126206,-0.677687,-0.472893,...,0,0,0,0,0,0,0,2.0,54,12


In [57]:
train1 = data12[~data12.Labels.isna()]
test1 = data12[data12.Labels.isna()]
test1.drop("Labels",axis=1,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [58]:
X=train1.drop(['Labels'],axis=1)
y=train1['Labels']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [60]:
vclf.fit(X_train,y_train)

Learning rate set to 0.099662


0:	learn: 1.7801064	total: 24.9ms	remaining: 24.9s
1:	learn: 1.7700998	total: 44.6ms	remaining: 22.3s
2:	learn: 1.7627282	total: 64.7ms	remaining: 21.5s
3:	learn: 1.7543507	total: 85ms	remaining: 21.2s
4:	learn: 1.7476443	total: 104ms	remaining: 20.7s
5:	learn: 1.7403276	total: 125ms	remaining: 20.6s
6:	learn: 1.7333950	total: 145ms	remaining: 20.5s
7:	learn: 1.7269331	total: 165ms	remaining: 20.4s
8:	learn: 1.7205727	total: 189ms	remaining: 20.8s
9:	learn: 1.7146667	total: 208ms	remaining: 20.6s
10:	learn: 1.7101807	total: 227ms	remaining: 20.4s
11:	learn: 1.7048148	total: 246ms	remaining: 20.2s
12:	learn: 1.6996881	total: 265ms	remaining: 20.1s
13:	learn: 1.6961559	total: 283ms	remaining: 19.9s
14:	learn: 1.6926939	total: 302ms	remaining: 19.8s
15:	learn: 1.6879234	total: 321ms	remaining: 19.7s
16:	learn: 1.6832000	total: 340ms	remaining: 19.7s
17:	learn: 1.6784334	total: 359ms	remaining: 19.6s
18:	learn: 1.6739606	total: 378ms	remaining: 19.5s
19:	learn: 1.6700990	total: 397ms	remai

161:	learn: 1.3239359	total: 3.14s	remaining: 16.2s
162:	learn: 1.3215599	total: 3.16s	remaining: 16.2s
163:	learn: 1.3197014	total: 3.18s	remaining: 16.2s
164:	learn: 1.3177792	total: 3.19s	remaining: 16.2s
165:	learn: 1.3162203	total: 3.21s	remaining: 16.1s
166:	learn: 1.3138569	total: 3.23s	remaining: 16.1s
167:	learn: 1.3120147	total: 3.25s	remaining: 16.1s
168:	learn: 1.3105608	total: 3.27s	remaining: 16.1s
169:	learn: 1.3086690	total: 3.29s	remaining: 16s
170:	learn: 1.3068928	total: 3.3s	remaining: 16s
171:	learn: 1.3050315	total: 3.32s	remaining: 16s
172:	learn: 1.3036220	total: 3.34s	remaining: 16s
173:	learn: 1.3016343	total: 3.36s	remaining: 16s
174:	learn: 1.3002704	total: 3.38s	remaining: 15.9s
175:	learn: 1.2983577	total: 3.4s	remaining: 15.9s
176:	learn: 1.2962803	total: 3.42s	remaining: 15.9s
177:	learn: 1.2943205	total: 3.44s	remaining: 15.9s
178:	learn: 1.2929560	total: 3.46s	remaining: 15.9s
179:	learn: 1.2904060	total: 3.48s	remaining: 15.8s
180:	learn: 1.2885715	to

320:	learn: 1.0547350	total: 6.29s	remaining: 13.3s
321:	learn: 1.0535771	total: 6.31s	remaining: 13.3s
322:	learn: 1.0519349	total: 6.33s	remaining: 13.3s
323:	learn: 1.0501134	total: 6.34s	remaining: 13.2s
324:	learn: 1.0484562	total: 6.36s	remaining: 13.2s
325:	learn: 1.0464846	total: 6.38s	remaining: 13.2s
326:	learn: 1.0446552	total: 6.4s	remaining: 13.2s
327:	learn: 1.0428238	total: 6.42s	remaining: 13.2s
328:	learn: 1.0412378	total: 6.44s	remaining: 13.1s
329:	learn: 1.0399176	total: 6.46s	remaining: 13.1s
330:	learn: 1.0391223	total: 6.47s	remaining: 13.1s
331:	learn: 1.0382266	total: 6.49s	remaining: 13.1s
332:	learn: 1.0371299	total: 6.51s	remaining: 13s
333:	learn: 1.0357555	total: 6.53s	remaining: 13s
334:	learn: 1.0341842	total: 6.55s	remaining: 13s
335:	learn: 1.0329170	total: 6.57s	remaining: 13s
336:	learn: 1.0311764	total: 6.58s	remaining: 13s
337:	learn: 1.0296510	total: 6.6s	remaining: 12.9s
338:	learn: 1.0279511	total: 6.62s	remaining: 12.9s
339:	learn: 1.0260643	to

479:	learn: 0.8454018	total: 10s	remaining: 10.9s
480:	learn: 0.8439604	total: 10s	remaining: 10.8s
481:	learn: 0.8427698	total: 10.1s	remaining: 10.8s
482:	learn: 0.8419788	total: 10.1s	remaining: 10.8s
483:	learn: 0.8406880	total: 10.1s	remaining: 10.8s
484:	learn: 0.8393832	total: 10.1s	remaining: 10.7s
485:	learn: 0.8383228	total: 10.1s	remaining: 10.7s
486:	learn: 0.8371622	total: 10.2s	remaining: 10.7s
487:	learn: 0.8362041	total: 10.2s	remaining: 10.7s
488:	learn: 0.8349865	total: 10.2s	remaining: 10.7s
489:	learn: 0.8339047	total: 10.2s	remaining: 10.6s
490:	learn: 0.8327387	total: 10.2s	remaining: 10.6s
491:	learn: 0.8318714	total: 10.3s	remaining: 10.6s
492:	learn: 0.8307536	total: 10.3s	remaining: 10.6s
493:	learn: 0.8297245	total: 10.3s	remaining: 10.6s
494:	learn: 0.8281696	total: 10.3s	remaining: 10.5s
495:	learn: 0.8267897	total: 10.4s	remaining: 10.5s
496:	learn: 0.8252806	total: 10.4s	remaining: 10.5s
497:	learn: 0.8239645	total: 10.4s	remaining: 10.5s
498:	learn: 0.82

638:	learn: 0.6798224	total: 13s	remaining: 7.34s
639:	learn: 0.6788452	total: 13s	remaining: 7.32s
640:	learn: 0.6778873	total: 13s	remaining: 7.3s
641:	learn: 0.6769009	total: 13.1s	remaining: 7.28s
642:	learn: 0.6760159	total: 13.1s	remaining: 7.26s
643:	learn: 0.6753878	total: 13.1s	remaining: 7.24s
644:	learn: 0.6742139	total: 13.1s	remaining: 7.21s
645:	learn: 0.6732283	total: 13.1s	remaining: 7.19s
646:	learn: 0.6723610	total: 13.1s	remaining: 7.17s
647:	learn: 0.6711346	total: 13.2s	remaining: 7.15s
648:	learn: 0.6700027	total: 13.2s	remaining: 7.13s
649:	learn: 0.6690489	total: 13.2s	remaining: 7.11s
650:	learn: 0.6684261	total: 13.2s	remaining: 7.09s
651:	learn: 0.6673081	total: 13.2s	remaining: 7.07s
652:	learn: 0.6665121	total: 13.3s	remaining: 7.04s
653:	learn: 0.6655248	total: 13.3s	remaining: 7.02s
654:	learn: 0.6643378	total: 13.3s	remaining: 7s
655:	learn: 0.6636030	total: 13.3s	remaining: 6.98s
656:	learn: 0.6629790	total: 13.3s	remaining: 6.96s
657:	learn: 0.6623490	

797:	learn: 0.5514914	total: 15.9s	remaining: 4.03s
798:	learn: 0.5510572	total: 15.9s	remaining: 4.01s
799:	learn: 0.5502742	total: 16s	remaining: 3.99s
800:	learn: 0.5495503	total: 16s	remaining: 3.97s
801:	learn: 0.5489034	total: 16s	remaining: 3.95s
802:	learn: 0.5484085	total: 16s	remaining: 3.93s
803:	learn: 0.5475467	total: 16s	remaining: 3.91s
804:	learn: 0.5468913	total: 16.1s	remaining: 3.89s
805:	learn: 0.5460276	total: 16.1s	remaining: 3.87s
806:	learn: 0.5451865	total: 16.1s	remaining: 3.85s
807:	learn: 0.5447770	total: 16.1s	remaining: 3.83s
808:	learn: 0.5438283	total: 16.1s	remaining: 3.81s
809:	learn: 0.5429565	total: 16.1s	remaining: 3.79s
810:	learn: 0.5420464	total: 16.2s	remaining: 3.77s
811:	learn: 0.5414033	total: 16.2s	remaining: 3.75s
812:	learn: 0.5406230	total: 16.2s	remaining: 3.73s
813:	learn: 0.5400234	total: 16.2s	remaining: 3.71s
814:	learn: 0.5394437	total: 16.2s	remaining: 3.69s
815:	learn: 0.5386567	total: 16.3s	remaining: 3.67s
816:	learn: 0.5380943	

956:	learn: 0.4538952	total: 19.3s	remaining: 866ms
957:	learn: 0.4534565	total: 19.3s	remaining: 846ms
958:	learn: 0.4527216	total: 19.3s	remaining: 826ms
959:	learn: 0.4522046	total: 19.3s	remaining: 806ms
960:	learn: 0.4517378	total: 19.4s	remaining: 785ms
961:	learn: 0.4510848	total: 19.4s	remaining: 765ms
962:	learn: 0.4503403	total: 19.4s	remaining: 745ms
963:	learn: 0.4499236	total: 19.4s	remaining: 725ms
964:	learn: 0.4493819	total: 19.4s	remaining: 705ms
965:	learn: 0.4487520	total: 19.4s	remaining: 684ms
966:	learn: 0.4482337	total: 19.5s	remaining: 664ms
967:	learn: 0.4476014	total: 19.5s	remaining: 644ms
968:	learn: 0.4469144	total: 19.5s	remaining: 624ms
969:	learn: 0.4464408	total: 19.5s	remaining: 604ms
970:	learn: 0.4458408	total: 19.5s	remaining: 583ms
971:	learn: 0.4451614	total: 19.6s	remaining: 563ms
972:	learn: 0.4447765	total: 19.6s	remaining: 543ms
973:	learn: 0.4443459	total: 19.6s	remaining: 523ms
974:	learn: 0.4437643	total: 19.6s	remaining: 503ms
975:	learn: 

VotingClassifier(estimators=[('lgg', LGBMClassifier(device='gpu', max_bin=25)),
                             ('xgg',
                              <catboost.core.CatBoostClassifier object at 0x7fb16feba2d0>)],
                 voting='soft')

In [61]:
prediction=vclf.predict_proba(test1)

In [62]:
# Results are using bert large sentence transformer
results3=pd.DataFrame(prediction, columns=[0,1,2,3,4,5])

In [63]:
final=(results+results1+results2+results3)/4

In [64]:
final.head()

,0,1,2,3,4,5
0,0.184829,0.198724,0.181477,0.197726,0.041716,0.195528
1,0.132969,0.233023,0.198231,0.259482,0.062737,0.113557
2,0.137865,0.228555,0.157320,0.125697,0.101759,0.248804
3,0.219476,0.319810,0.246131,0.110130,0.055268,0.049186
4,0.148455,0.282205,0.196449,0.113686,0.118126,0.141079


In [ ]:
final.to_excel('/kaggle/working/final.xlsx',index=False)